In [12]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import RepeatedKFold
path = os.getcwd()
os.chdir(os.path.join(path, 'e:/school/dpav/vizdom/src'))
from crawlers.url_crawlers import get_our_world_in_data, get_mask_data, get_expected_years_of_school_data, get_our_world_in_data_attributes
from filters.base_filters import country_based_interpolation, keep_columns_by_name, drop_rows_with_OWID, drop_rows_with_occurrence_number
from data_engine.base_engine import combine_data_with_mask, combine_data_with_expected_years_of_school, cap_outliers

In [13]:

attributes = get_our_world_in_data_attributes.keys()
covid_general = get_our_world_in_data()
covid_mask = get_mask_data()
expected_years_of_school = get_expected_years_of_school_data()

In [14]:
filtered_data = drop_rows_with_OWID(covid_general)
filtered_data = drop_rows_with_occurrence_number(filtered_data, 1)
interpolated_data = country_based_interpolation(filtered_data)
combined_data = combine_data_with_mask(interpolated_data, covid_mask, 'mask_use_mean')
combined_data = combine_data_with_expected_years_of_school(combined_data, expected_years_of_school)
remaining_data = keep_columns_by_name(combined_data, attributes)
remaining_data = cap_outliers(remaining_data, 'mask_use_mean')

In [15]:

data = remaining_data



# attributes = list(get_our_world_in_data_attributes.keys())
# attributes.remove('continent')
# attributes.remove('iso_code')
# attributes.remove('location')
# attributes.remove('date')

     iso_code continent     location       date  new_cases  total_cases  \
0         AFG      Asia  Afghanistan 2020-02-24        5.0          5.0   
1         AFG      Asia  Afghanistan 2020-02-25        0.0          5.0   
2         AFG      Asia  Afghanistan 2020-02-26        0.0          5.0   
3         AFG      Asia  Afghanistan 2020-02-27        0.0          5.0   
4         AFG      Asia  Afghanistan 2020-02-28        0.0          5.0   
...       ...       ...          ...        ...        ...          ...   
1010      AFG      Asia  Afghanistan 2022-11-30       77.0     205907.0   
1011      AFG      Asia  Afghanistan 2022-12-01      166.0     206073.0   
1012      AFG      Asia  Afghanistan 2022-12-02       60.0     206133.0   
1013      AFG      Asia  Afghanistan 2022-12-03       12.0     206145.0   
1014      AFG      Asia  Afghanistan 2022-12-04       61.0     206206.0   

      new_deaths  total_deaths  new_vaccinations  total_vaccinations  ...  \
0            1.0      

In [16]:

# train_data = norway.sample(frac=0.6, random_state=12)
# valid_and_test = norway.drop(train_data.index)
# valid_data = valid_and_test.sample(frac=0.5, random_state=62)
# test_data = valid_and_test.drop(valid_data.index)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 100)
# y = y.to_frame()

In [17]:
# y_train = pd.DataFrame(y_train)
# x_train = pd.DataFrame(x_train)
# x_test = pd.DataFrame(x_test)
# x_train_sort=x_train.sort_values(by = 'date')
# x_test_sort=x_test.sort_values(by = 'date')
# y_train_sort=y_train.sort_values(by = 'date')

In [18]:
# y_train_shift = y_train_sort.shift(-90, axis=0)


In [19]:
# number_of_nans = y_train_shift.isnull().values.ravel().sum()

# x_train_sort_dropped = x_train_sort.drop(x_train_sort.tail(number_of_nans).index)
# x_train_sort_dropped = x_train_sort.iloc[:-number_of_nans, :]
# y_train_shift_dropped = y_train_shift.dropna(axis=0)


In [20]:
# steps_ridge = [
#     ('scalar', StandardScaler()),
#     ('poly', PolynomialFeatures(degree=4)),
#     ('model', Ridge(alpha=10, fit_intercept=True))
# ]
# steps_lasso = [
#     ('scalar', StandardScaler()),
#     ('poly', PolynomialFeatures(degree=2)),
#     ('model', Lasso(alpha=0.3, fit_intercept=True))
# ]
# ridge_pipe = Pipeline(steps_ridge)
# lasso_pipe = Pipeline(steps_lasso)

# lasso_pipe.fit(x_train_sort_dropped, y_train_shift_dropped['new_cases'])
# ridge_pipe.fit(x_train_sort_dropped, y_train_shift_dropped['new_cases'])

# yhat0 = ridge_pipe.predict(x_train_sort)
# yhat1 = ridge_pipe.predict(x_train_sort)

In [21]:
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# lasso_cv = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
# lasso = Lasso(alpha=6.475e-7, max_iter=100000)
# linear = LinearRegression() 
# ridge = Ridge(alpha=6.475e-7, max_iter=100000, solver='sag')
# elastic = ElasticNet(alpha = 6.475e-7, max_iter=100000)
# lasso_cv.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases'])
# lasso.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases'])
# linear.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases'])
# elastic.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases'])
# ridge.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases']) 
# yhat0 = lasso_cv.predict(x_train_sort)
# yhat1 = lasso.predict(x_train_sort)
# yhat2 = linear.predict(x_train_sort)
# yhat3 = ridge.predict(x_train_sort)
# yhat4 = elastic.predict(x_train_sort)

In [22]:

# y_train_shift = y_train_shift.shift(90, freq='D')
# prediction_1 = pd.Series(yhat0)
# rolling_mean_1 = prediction_1.rolling(31).mean()
# prediction_2 = pd.Series(yhat1)
# rolling_mean_2 = prediction_2.rolling(60).mean()
# fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=yhat0,
#     line_color='cyan',
#     name='ridge',
# ))
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=yhat1,
#     line_color='green',
#     name='lasso',
# ))
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=rolling_mean_1,
#     line_color='red',
#     name='linear',
# ))
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=rolling_mean_2,
#     line_color='blue',
#     name='ridge',
# ))
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=yhat4,
#     line_color='orange',
#     name='elastic',
# ))

# fig.add_trace(go.Scatter(
#     x=y.index, y=y['new_cases'],
#     line_color='yellow',
#     name='original',
# ))

# fig.update_traces(mode='lines')
# fig
# plt.figure(figsize=(20, 10))
# plt.plot(x_train.index, y_train, 'b.',label = 'new death (train)')
# plt.plot(x_train.index, y_pred_mlr_train, 'r.',label = 'new death (prediction)')
# plt.legend()

# # plt.figure(figsize=(20, 10))
# plt.plot(x_test.index, y_test, 'bx',label = 'new death (test)')
# plt.plot(x_test.index, y_pred_mlr, 'rx',label = 'new death (prediction test)')
# plt.legend()
